In [1]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName("deva").getOrCreate()
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
#1.create RDD in 3 different ways.

In [11]:
demo="Welcome to spark".split()
spark.sparkContext.parallelize(demo,2).collect()

['Welcome', 'to', 'spark']

In [14]:
demo1=spark.sparkContext.textFile("textfile.txt")
demo1.collect()
type(demo1)

pyspark.rdd.RDD

In [ ]:
#2.Read a text file and count the number of words in the file using rdd operations.

In [21]:
demo_count=demo1.flatMap(lambda s:s.split())
demo_count.count()

5

In [ ]:
#3.write a program to find the word frequency in a given file.

In [22]:
info=spark.sparkContext.textFile("textfile.txt")
info=info.flatMap(lambda x:x.split())
content_map=info.map(lambda c:(c,1))
content_map.reduceByKey(lambda a,b:a+b).collect()

[('hello', 1), ('deva', 1), ('welcome', 1), ('to', 1), ('spark', 1)]

In [ ]:
#4.write a program to convert all words in a file to uppercase.

In [25]:
demo1.map(lambda c:c.upper()).collect()

['hello deva welcome to spark']

In [ ]:
#5.write a program to convert all words in a file to lowercase.

In [28]:
demo1.map(lambda c:c.lower())
demo1.collect()

['hello deva welcome to spark']

In [ ]:
#6.write a program to capitalize first letter of each word in file use string capaitalize() method.

In [29]:
capitalize=demo1.flatMap(lambda a:a.split(" ")).map(lambda c:c.capitalize()).collect()
" ".join(capitalize)

'Hello Deva Welcome To Spark'

In [ ]:
#7.find the longest length of word from given set of words.

In [39]:
long_word=demo1.flatMap(lambda x:x.split(" "))
long_word.map(lambda n:(len(n),n)).max()[1]

'welcome'

In [ ]:
#8.Map the Registration numbers to corresponding branch. 6000 series BDA, 9000 series HAD, 1000 series MS, 2000 series VLSI, 3000 series ES, 4000 series MSc, 5000 series CC.

In [52]:
reg_num=[6001,2005,6035,6021,9004,9056,3088,3045,4088,4065,5005,5016,2001,2002]
context=spark.sparkContext.parallelize(reg_num,2)
classify=context.map(lambda reg:('VLSI',reg) if reg>2000 and reg<3000 
        else ('MS',reg) if reg>1000 and reg<2000
        else ('ES',reg) if reg>3000 and reg<4000
        else ('MSc',reg) if reg>4000 and reg<5000
        else ('CC',reg) if reg>5000 and reg<6000
        else ('BDA',reg) if reg>6000 and reg<7000
        else ('HDA',reg))
classify.collect()

[('BDA', 6001),
 ('VLSI', 2005),
 ('BDA', 6035),
 ('BDA', 6021),
 ('HDA', 9004),
 ('HDA', 9056),
 ('ES', 3088),
 ('ES', 3045),
 ('MSc', 4088),
 ('MSc', 4065),
 ('CC', 5005),
 ('CC', 5016),
 ('VLSI', 2001),
 ('VLSI', 2002)]

In [ ]:
#9.given registration number generate a key value pair of registration number and corresponding branch.

In [46]:
reg_no=[6027]
info=spark.sparkContext.parallelize(reg_no,2)
ans=info.map(lambda reg:('VLSI',reg) if reg>2000 and reg<3000 
        else ('MS',reg) if reg>1000 and reg<2000
        else ('ES',reg) if reg>3000 and reg<4000
        else ('MSc',reg) if reg>4000 and reg<5000
        else ('CC',reg) if reg>5000 and reg<6000
        else ('BDA',reg) if reg>6000 and reg<7000
        else ('HDA',reg))
ans.collect()

[('BDA', 6027)]

In [ ]:
#10.A text file contains data about citizens of country. Fields(information in file) are Name, dob, Phone, email and state name. Another file contains mapping of state names to state code like Karnataka is codes as KA, TamilNadu as TN, Kerala KL etc. Compress the file will by changing full state name to state code.

In [76]:
data=spark.sparkContext.textFile("userdata.txt")
code=spark.sparkContext.textFile("statecode.txt")
data_rdd=data.map(lambda x:x.split(",")).collect()
code_rdd=code.map(lambda y:y.split(",")).collect()
for i in range(len(data_rdd)):
    for j in range(len(code_rdd)):  
        if data_rdd[i][4]==code_rdd[j][0]:
            data_rdd[i][4]=code_rdd[j][1]
data_rdd

[['deva', '26-06-1996', '12345', 'deva@gmail.com', 'Karanataka'],
 ['surya', '26-01-1996', '23456', 'surya@gmail.com', 'AP'],
 ['sainath', '26-06-1996', '45678', 'sainath@gmail.com', 'TN'],
 ['vamsi', '4-01-1996', '78945', 'vamsi@gmsil.com', 'GJ'],
 ['sriakr', '24-02-1996', '45678', 'srikar@gmail.com', 'JK'],
 ['naren', '11-08-1992', '78912', 'naren@gmail.com', 'WB'],
 ['ajith', '16-05-1995', '12456', 'ajith@gmail.com', 'PB'],
 ['adlen', '11-11-1996', '36987', 'adlen@gmail.com', 'MH']]

In [77]:
code_dict = {}
for val in stateKey.collect():
    code_dict[val[0]] = val[1]
    
code_dict

{'AndhraPradesh': 'AP',
 'Karnataka': 'KA',
 'JammuKashmir': 'JK',
 'Gujarat': 'GJ',
 'Maharashtra': 'MH',
 'Rajasthan': 'RJ',
 'Punjab': 'PB',
 'TamilNadu': 'TN',
 'UttarPradesh': 'UP',
 'WestBengal': 'WB'}

In [78]:
mapData = spark.sparkContext.broadcast(code_dict)

cityRdd = spark.sparkContext.textFile('userdata.txt')
print(cityRdd.collect())
def abc(state,codes):
    splitData = state.split(',')  
    print(splitData)
    splitData[4] = codes.value.get(splitData[4])
    newData = ' '
    newData = newData.join(splitData)
    
    return newData
    
mapCitizen = cityRdd.map(lambda data: abc(data,mapData))
mapCitizen.collect()

['deva,26-06-1996,12345,deva@gmail.com,Karanataka', 'surya,26-01-1996,23456,surya@gmail.com,AndhraPradesh', 'sainath,26-06-1996,45678,sainath@gmail.com,TamilNadu', 'vamsi,4-01-1996,78945,vamsi@gmsil.com,Gujarat', 'sriakr,24-02-1996,45678,srikar@gmail.com,JammuKashmir', 'naren,11-08-1992,78912,naren@gmail.com,WestBengal', 'ajith,16-05-1995,12456,ajith@gmail.com,Punjab', 'adlen,11-11-1996,36987,adlen@gmail.com,Maharashtra']


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 86, 192.168.0.10, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-3.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 605, in main
  File "C:\spark-3.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 597, in process
  File "C:\spark-3.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark-3.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-78-3d7d28281d31>", line 14, in <lambda>
  File "<ipython-input-78-3d7d28281d31>", line 10, in abc
TypeError: s

In [ ]:
#11.Text file contain numbers. Numbers are separated by one white space. There is no order to store the numbers. One line may contain one or more numbers.Find the maximum, minimum, sum and mean of numbers.

In [47]:
demo2=spark.sparkContext.textFile("number.txt")
demo2_rdd=demo2.flatMap(lambda z:z.split(" ")).map(lambda c:int(c))

In [48]:
demo2_rdd.max()

101

In [49]:
demo2_rdd.min()

9

In [50]:
demo2_rdd.sum()

1371

In [51]:
demo2_rdd.mean()

48.964285714285715